In [76]:
# import dependencies
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
from config import Password, Username, Host,DB
import pandas as pd

In [77]:
# connect to database
engine = create_engine(f'postgresql://{Username}:{Password}@{Host}:5432/{DB}', echo=False)
connection = engine.connect()
inspector = inspect(engine)
inspector.get_table_names()

['Person',
 'Movie_Genre_Junction',
 'Movie',
 'Role',
 'Genre',
 'Movie_Person_Role_Junction']

In [78]:
# fetch data from database
cursor = engine.execute('SELECT public."Movie"."movie_id", public."Movie"."movie_rating", public."Movie"."release_date", public."Movie"."runtime", public."Movie"."metascore", \
    public."Movie"."imdb_rating", public."Movie"."boxoffice", public."Movie"."budget", public."Movie"."buzz"  \
    FROM public."Movie"').fetchall()

In [79]:
movies_df = pd.DataFrame(cursor, columns=["movie_id", "rating", "date", "runtime", "metascore", "imdb_rating", "boxoffice", "budget", "buzz"])
movies_df

,movie_id,rating,date,runtime,metascore,imdb_rating,boxoffice,budget,buzz
0,1,PG,2006-12-22,108,48,6.4,250863268,110000000,6
1,2,G,2006-06-09,117,73,7.1,244082982,70000000,4
2,3,PG-13,2006-05-26,104,58,6.7,234362462,210000000,23
3,4,PG-13,2006-05-19,149,46,6.6,217536138,125000000,7
4,5,PG-13,2006-06-28,154,72,6.0,200120000,232000000,15
...,...,...,...,...,...,...,...,...,...
1662,6800,TV-MA,2021-02-12,88,44,5.5,892802,750000,4
1663,6803,R,2021-05-21,117,69,6.9,845464,1400000,24
1664,6835,PG-13,2021-05-07,108,67,6.4,632542,15000000,1
1665,6850,Not Rated,2021-06-18,101,65,6.9,546201,7000000,4


In [47]:
# TEST QUERY
# cursor = engine.execute('SELECT public."Movie"."movie_id", public."Movie"."movie_rating", public."Movie"."release_date", public."Movie"."runtime", public."Movie"."metascore", \
#     public."Movie"."imdb_rating", public."Movie"."boxoffice", public."Movie"."budget", public."Movie"."buzz", public."Genre"."genre_name", concat(public."Person"."name",  public."Role"."role") \
#     FROM public."Movie" \
#     LEFT JOIN public."Movie_Genre_Junction" ON public."Movie"."movie_id" = public."Movie_Genre_Junction"."movie_id" \
#     LEFT JOIN public."Genre" ON public."Movie_Genre_Junction"."genre_id" = public."Genre"."genre_id" \
#     LEFT JOIN public."Movie_Person_Role_Junction" ON public."Movie"."movie_id" = public."Movie_Person_Role_Junction"."movie_id" \
#     LEFT JOIN public."Role" ON public."Role"."role_id" = public."Movie_Person_Role_Junction"."role_id" \
#     LEFT JOIN public."Person" ON public."Person"."person_id" = public."Movie_Person_Role_Junction"."person_id" \
#     ').fetchall()

In [58]:
# TEST QUERY
# cursor = engine.execute('SELECT concat(public."Person"."name",  public."Role"."role") \
#     FROM public."Movie" \
#     LEFT JOIN public."Movie_Person_Role_Junction" ON public."Movie"."movie_id" = public."Movie_Person_Role_Junction"."movie_id" \
#     LEFT JOIN public."Role" ON public."Role"."role_id" = public."Movie_Person_Role_Junction"."role_id" \
#     LEFT JOIN public."Person" ON public."Person"."person_id" = public."Movie_Person_Role_Junction"."person_id" \
#     WHERE public."Movie"."movie_id" = 75 \
#     ').fetchall()

In [50]:
# TESTING
# movies_df[cursor[0][0]] = 0
# movies_df.loc[movies_df["movie_id"] == 1, [cursor[1][0]]] = 1

In [62]:
# TESTING
# if cursor[0][0] in movies_df:
#     print("exists")

exists


In [74]:
# TESTING
# genres = engine.execute('SELECT public."Genre"."genre_name"\
#     FROM public."Movie" \
#     LEFT JOIN public."Movie_Genre_Junction" ON public."Movie"."movie_id" = public."Movie_Genre_Junction"."movie_id" \
#     LEFT JOIN public."Genre" ON public."Movie_Genre_Junction"."genre_id" = public."Genre"."genre_id" \
#     WHERE public."Movie"."movie_id" = 75').fetchall()

In [81]:
for index, row in movies_df.iterrows():
    # make sql call for people
    people_query = engine.execute(f'SELECT concat(public."Person"."name",  public."Role"."role") \
        FROM public."Movie" \
        LEFT JOIN public."Movie_Person_Role_Junction" ON public."Movie"."movie_id" = public."Movie_Person_Role_Junction"."movie_id" \
        LEFT JOIN public."Role" ON public."Role"."role_id" = public."Movie_Person_Role_Junction"."role_id" \
        LEFT JOIN public."Person" ON public."Person"."person_id" = public."Movie_Person_Role_Junction"."person_id" \
        WHERE public."Movie"."movie_id" = {row["movie_id"]} \
        ').fetchall()
    for person in people_query:
        # if person_role column doesn't exist, create it with default value of 0
        if person[0] not in movies_df:
            movies_df[person[0]] = 0
        # update current row to include person_role
        movies_df.loc[movies_df["movie_id"] == row["movie_id"], [person[0]]] = 1
    
    # make sql call for genres
    genre_query = engine.execute(f'SELECT public."Genre"."genre_name"\
        FROM public."Movie" \
        LEFT JOIN public."Movie_Genre_Junction" ON public."Movie"."movie_id" = public."Movie_Genre_Junction"."movie_id" \
        LEFT JOIN public."Genre" ON public."Movie_Genre_Junction"."genre_id" = public."Genre"."genre_id" \
        WHERE public."Movie"."movie_id" = {row["movie_id"]}').fetchall()
    for genre in genre_query:
        if genre[0] not in movies_df:
            movies_df[genre[0]] = 0
        movies_df.loc[movies_df["movie_id"] == row["movie_id"], [genre[0]]] = 1
    

In [82]:
movies_df

,movie_id,rating,date,runtime,metascore,imdb_rating,boxoffice,budget,buzz,Shawn LevyDirector,...,Aidan ChambersWriter,Félix LefebvreActor,Benjamin VoisinActor,Philippine VelgeActor,Anthony Scott BurnsDirector,Anthony Scott BurnsWriter,Daniel WeissenbergerWriter,Julia Sarah StoneActor,Landon LiboironActor,Carlee RyskiActor
0,1,PG,2006-12-22,108,48,6.4,250863268,110000000,6,1,...,0,0,0,0,0,0,0,0,0,0
1,2,G,2006-06-09,117,73,7.1,244082982,70000000,4,0,...,0,0,0,0,0,0,0,0,0,0
2,3,PG-13,2006-05-26,104,58,6.7,234362462,210000000,23,0,...,0,0,0,0,0,0,0,0,0,0
3,4,PG-13,2006-05-19,149,46,6.6,217536138,125000000,7,0,...,0,0,0,0,0,0,0,0,0,0
4,5,PG-13,2006-06-28,154,72,6.0,200120000,232000000,15,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1662,6800,TV-MA,2021-02-12,88,44,5.5,892802,750000,4,0,...,0,0,0,0,0,0,0,0,0,0
1663,6803,R,2021-05-21,117,69,6.9,845464,1400000,24,0,...,0,0,0,0,0,0,0,0,0,0
1664,6835,PG-13,2021-05-07,108,67,6.4,632542,15000000,1,0,...,0,0,0,0,0,0,0,0,0,0
1665,6850,Not Rated,2021-06-18,101,65,6.9,546201,7000000,4,0,...,1,1,1,1,0,0,0,0,0,0


In [84]:
# save csv so we don't have to run the loop again
movies_df.to_csv("dummy_data_frame.csv")

# Preprocess Data

In [89]:
# separate out target: Box Office
target = movies_df["boxoffice"]
movies_df.drop(columns=["boxoffice"], inplace=True)

In [87]:
# clean data
# date into month
movies_df["month"] = pd.DatetimeIndex(movies_df['date']).month


In [90]:
# make dummies for rating
dummy_df = pd.get_dummies(movies_df, columns=["rating"])

In [92]:
dummy_df.drop(columns=["movie_id", "date"], inplace=True)

In [93]:
dummy_df

,runtime,metascore,imdb_rating,budget,buzz,Shawn LevyDirector,Robert Ben GarantWriter,Thomas LennonWriter,Milan TrencWriter,Ben StillerActor,...,rating_G,rating_NC-17,rating_Not Rated,rating_PG,rating_PG-13,rating_R,rating_TV-14,rating_TV-MA,rating_TV-PG,rating_Unrated
0,108,48,6.4,110000000,6,1,1,1,1,1,...,0,0,0,1,0,0,0,0,0,0
1,117,73,7.1,70000000,4,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,104,58,6.7,210000000,23,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,149,46,6.6,125000000,7,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,154,72,6.0,232000000,15,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1662,88,44,5.5,750000,4,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1663,117,69,6.9,1400000,24,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1664,108,67,6.4,15000000,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1665,101,65,6.9,7000000,4,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [ ]:
# scale data


In [ ]:
# split data into test train
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Create Models

In [ ]:
# linear regression 
# remove noisy data first
# keep it simple 
# Import LinearRegression class
from pyspark.ml.regression import LinearRegression

# Define LinearRegression algorithm
lr = LinearRegression()

# Tweak regParam
modelA = lr.fit(data, {lr.regParam:0.0})

In [ ]:
# svc
from sklearn.svm import SVC
svm_model = SVC(kernel='linear')
model.fit(X_train, y_train)

In [ ]:
# SVC Model Accuracy
print('Test Acc: %.3f' % model.score(X_test, y_test))

In [ ]:
# SVC predictions
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=target_names))

In [ ]:
# random forest
# need to bin data for classifications
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=1, n_estimators=500).fit(X_train_scaled, y_train)

In [ ]:
# random forest feature importances
features = clf.feature_importances_
print(features)
plt.bar(x = range(len(features)), height=features)
plt.show()

In [ ]:
# neural network
import tensorflow as tf
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=5, activation="relu", input_dim=2))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
nn_model.summary()

In [ ]:
# neural net
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# neural net
# Fit the model to the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

In [ ]:
# xg boost
from xgboost import XGBClassifier

# fit model no training data
xg_model = XGBClassifier()
xg_model.fit(X_train, y_train)


In [ ]:
# xg boost
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

# Output Models